In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importation de l'input a partir de la dataset

In [ ]:
df = pd.read_csv('../input/voicegender/voice.csv')

Affichage des 10 premiers elements des données

In [ ]:
df.head(10)

* Affichage des quelque elements de meanfreq à modindx

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.meanfreq.value_counts()

Visualisation

In [ ]:
#Printing the total distribution

print("Total Number of samples : {}".format(df.shape[0]))

print("Total No.of Males : {}".format(df[df.label == 'male'].shape[0]))

print("Total No.of Females : {}".format(df[df.label == 'female'].shape[0]))

In [ ]:
#Checking for Null Values

df.isnull().sum()

In [ ]:
df.head()
df.plot(kind='scatter', x='meanfreq', y='dfrange')
df.plot(kind='kde', y='meanfreq')

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
sns.pairplot(df[['meanfreq', 'Q25', 'Q75', 'skew', 'centroid', 'label']], 
                 hue='label', size=2)

In [ ]:
sns.jointplot("meanfreq", "Q25", df, kind='kde');

In [ ]:
plt.figure(figsize=(12,12))
sns.kdeplot(df.meanfreq, df.Q25,  shade=True)

In [ ]:
sns.boxplot(x="meanfreq", y="Q25", data=df)

In [ ]:
plt.figure(figsize=(9,9))

logistique = lambda x: np.exp(x)/(1+np.exp(x))   

x_range = np.linspace(-10,10,50)       
y_values = logistique(x_range)

plt.plot(x_range, y_values, color="black")

separation des données

Resumé du modele

Définition des parameters

In [ ]:
df.label = [1 if each == "male" else 0 for each in df.label]

df.head()

In [ ]:
X = df.drop(["label"], axis = 1)
y = df.label.values

#Normalizing X

X = (X - X.min()) / (X.max() - X.min())

print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
model = keras.Sequential([keras.layers.InputLayer(input_shape=X_train.shape[1:]),
                          keras.layers.Dense(32, activation='relu'),
                          keras.layers.Dense(64, activation='relu'),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(32, activation='relu'),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(16, activation='relu'),
                          keras.layers.Dense(1, activation='sigmoid')
                         ])

Resumé modele


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

v_split= 0.2
epoch = 100

Modele


In [ ]:

train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

Performance du modele

In [ ]:
history_df = pd.DataFrame(history.history)

history_df.plot(figsize=(10,10))

plt.grid(True)
plt.gca().set_ylim(0,1.15)
plt.show()

Resultat

In [ ]:
# Test
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))